In [1]:
import json

#create the docs_raw variable from the .json file
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

documents = []


#siccome elastic search vuole tutti i documenti su un livello, mettiamo ad ogni documento il suo corso
#prima i documenti erano suddivisi in sezione, in base al corso
#guarda le differenze tra docs_raw e documents
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [ ]:
documents

In [ ]:
pip install sentence_transformers==2.7.0

In [4]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer("all-mpnet-base-v2")